In [1]:
import matplotlib.pyplot as plt

# 设置 matplotlib 使用支持中文的字体
# 'Microsoft YaHei' 是 Windows 系统中常见的字体
# 如果您使用 macOS，可以尝试 'PingFang SC'
# 如果您使用 Linux，可以尝试 'WenQuanYi Micro Hei'
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei'] 

# 解决负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False 


# Jupyter笔记本中的可共享性问题

这个笔记本演示了Jupyter在共享和部署方面的困难：
1. 难以部署为Web应用
2. 交互性在静态分享中丢失
3. 需要复杂的基础设施

## 问题1：无法直接部署为Web应用

Jupyter笔记本主要设计用于开发环境，不是为最终用户设计的

In [3]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 创建一个交互式数据分析界面

In [4]:
# 数据生成参数
sample_size = widgets.IntSlider(
    value=100,
    min=50,
    max=1000,
    step=50,
    description='样本大小:'
)

noise_level = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1,
    step=0.05,
    description='噪声水平:'
)

chart_type = widgets.Dropdown(
    options=['散点图', '直方图', '箱线图'],
    value='散点图',
    description='图表类型:'
)

display(sample_size, noise_level, chart_type)

IntSlider(value=100, description='样本大小:', max=1000, min=50, step=50)

FloatSlider(value=0.1, description='噪声水平:', max=1.0, step=0.05)

Dropdown(description='图表类型:', options=('散点图', '直方图', '箱线图'), value='散点图')

In [5]:
# 输出区域
output = widgets.Output()

def update_analysis(change=None):
    with output:
        output.clear_output(wait=True)
        
        # 生成数据
        size = sample_size.value
        noise = noise_level.value
        
        np.random.seed(42)  # 为了可重现性
        x = np.linspace(0, 10, size)
        y = 2 * x + 1 + np.random.normal(0, noise * 5, size)
        
        data = pd.DataFrame({'X': x, 'Y': y})
        
        # 创建图表
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        chart = chart_type.value
        if chart == '散点图':
            ax1.scatter(data['X'], data['Y'], alpha=0.6)
            ax1.set_xlabel('X')
            ax1.set_ylabel('Y')
            ax1.set_title('X vs Y 散点图')
        elif chart == '直方图':
            ax1.hist(data['Y'], bins=20, alpha=0.7)
            ax1.set_xlabel('Y值')
            ax1.set_ylabel('频次')
            ax1.set_title('Y值分布直方图')
        else:  # 箱线图
            ax1.boxplot([data['X'], data['Y']], labels=['X', 'Y'])
            ax1.set_title('X和Y的箱线图')
        
        # 统计信息
        ax2.axis('off')
        stats_text = f"""
        数据统计信息:
        
        样本大小: {size}
        噪声水平: {noise:.2f}
        
        X统计:
        均值: {data['X'].mean():.2f}
        标准差: {data['X'].std():.2f}
        
        Y统计:
        均值: {data['Y'].mean():.2f}
        标准差: {data['Y'].std():.2f}
        
        相关系数: {data['X'].corr(data['Y']):.3f}
        """
        ax2.text(0.1, 0.9, stats_text, transform=ax2.transAxes, 
                fontsize=10, verticalalignment='top')
        
        plt.tight_layout()
        plt.show()

# 绑定事件
sample_size.observe(update_analysis, names='value')
noise_level.observe(update_analysis, names='value')
chart_type.observe(update_analysis, names='value')

# 初始显示
update_analysis()

display(output)

Output()

### 添加数据导出功能

In [ ]:
export_button = widgets.Button(
    description='导出数据',
    button_style='success'
)

export_output = widgets.Output()

def export_data(button):
    with export_output:
        export_output.clear_output(wait=True)
        
        # 重新生成当前数据
        size = sample_size.value
        noise = noise_level.value
        
        np.random.seed(42)
        x = np.linspace(0, 10, size)
        y = 2 * x + 1 + np.random.normal(0, noise * 5, size)
        
        data = pd.DataFrame({'X': x, 'Y': y})
        
        # 在实际应用中，这里会保存到文件
        print(f"数据已导出 (模拟)")
        print(f"文件名: data_sample_{size}_noise_{noise:.2f}.csv")
        print(f"数据预览:")
        print(data.head())

export_button.on_click(export_data)

display(export_button, export_output)

**问题说明**：

上面创建了一个有用的交互式数据分析工具，但是：

1. **无法直接部署**：不能简单地将这个笔记本部署为Web应用供其他人使用
2. **需要Jupyter环境**：用户必须安装并运行Jupyter才能使用
3. **复杂的部署流程**：需要使用Voilà、Binder或其他工具，配置复杂
4. **性能问题**：在Web部署时性能通常不佳
5. **安全考虑**：直接暴露Jupyter环境存在安全风险

## 问题2：静态分享丢失交互性

In [ ]:
# 这个交互式组件在静态分享中完全无用
interactive_demo = widgets.interact(
    lambda freq, amplitude: plt.plot(
        np.linspace(0, 2*np.pi, 100),
        amplitude * np.sin(freq * np.linspace(0, 2*np.pi, 100))
    ) or plt.title(f'正弦波: 频率={freq}, 振幅={amplitude}') or plt.show(),
    freq=widgets.FloatSlider(min=0.5, max=5, step=0.1, value=1),
    amplitude=widgets.FloatSlider(min=0.1, max=2, step=0.1, value=1)
)

**静态分享的问题**：

1. **GitHub渲染**：在GitHub上查看时，所有交互元素都不工作
2. **nbviewer**：即使使用nbviewer，交互性也会丢失
3. **PDF导出**：转换为PDF时只能看到静态快照
4. **邮件分享**：通过邮件分享的HTML版本没有交互性

用户只能看到代码和最后一次运行的输出，无法体验交互功能。

## 问题3：复杂的部署基础设施

**要将Jupyter笔记本部署为Web应用，需要**：

### 选项1：使用Voilà
```bash
pip install voila
voila notebook.ipynb --port=8866
```
**问题**：
- 需要额外安装和配置
- 性能不佳
- 定制化困难
- 错误处理不友好

### 选项2：使用Binder
**问题**：
- 启动时间长（几分钟）
- 资源限制
- 不适合生产环境
- 依赖外部服务

### 选项3：自建JupyterHub
**问题**：
- 复杂的基础设施要求
- 安全配置困难
- 维护成本高
- 扩展性问题

### 选项4：转换为其他框架
**问题**：
- 需要重写代码（如转换为Streamlit、Dash等）
- 丢失原有的开发体验
- 维护两套代码

## 总结

Jupyter笔记本在可共享性方面的主要问题：

1. **部署复杂**：需要额外工具和复杂配置
2. **交互性丢失**：静态分享无法保持交互功能
3. **性能问题**：Web部署时性能通常不佳
4. **安全风险**：直接暴露开发环境
5. **维护困难**：需要管理复杂的部署基础设施
6. **用户体验差**：最终用户需要技术知识才能使用

这些问题使得Jupyter笔记本主要适合开发和探索，而不适合作为最终产品分享给非技术用户。